In [1]:
import sys
import os
from pathlib import Path
import pickle
import torch
import pprint
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')

In [2]:
args = parser.parse_args([])
args.task = 'normal'
args.metric = 'nominal'
args.SoftEva = True
args = FormulateArgs(args)

In [3]:
results = torch.zeros([13,10,4])

for seed in range(10):
    for ds in range(13):
        args.SEED = seed
        args.DATASET = ds

        valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset')
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset')

        modelname = f"model_pNN_data_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
        
        
        if os.path.isfile(f'./models/{modelname}'):                
            print(modelname)
            
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            model.SetParameter('args', args)

            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            acc_valid, power_valid = evaluator(model, X_valid, y_valid)
            acc_test, power_test   = evaluator(model, X_test,  y_test)

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
            results[ds,seed,2] = power_valid
            results[ds,seed,3] = power_test
            
        else:
            results[ds,seed,:] = float('nan')

model_pNN_data_acuteinflammation_seed_00.model
model_pNN_data_balancescale_seed_00.model
model_pNN_data_breastcancerwisc_seed_00.model
model_pNN_data_cardiotocography3clases_seed_00.model
model_pNN_data_energyy1_seed_00.model
model_pNN_data_energyy2_seed_00.model
model_pNN_data_iris_seed_00.model
model_pNN_data_mammographic_seed_00.model
model_pNN_data_pendigits_seed_00.model
model_pNN_data_seeds_seed_00.model
model_pNN_data_tictactoe_seed_00.model
model_pNN_data_vertebralcolumn2clases_seed_00.model
model_pNN_data_vertebralcolumn3clases_seed_00.model
model_pNN_data_acuteinflammation_seed_01.model
model_pNN_data_balancescale_seed_01.model
model_pNN_data_breastcancerwisc_seed_01.model
model_pNN_data_cardiotocography3clases_seed_01.model
model_pNN_data_energyy1_seed_01.model
model_pNN_data_energyy2_seed_01.model
model_pNN_data_iris_seed_01.model
model_pNN_data_mammographic_seed_01.model
model_pNN_data_pendigits_seed_01.model
model_pNN_data_seeds_seed_01.model
model_pNN_data_tictactoe_seed

In [4]:
results.shape

torch.Size([13, 10, 4])

In [5]:
valid_acc_data = results[:, :, 0]
best_seeds = torch.argmax(valid_acc_data, dim=1)
best_seeds

tensor([0, 3, 5, 0, 2, 3, 0, 0, 9, 6, 9, 0, 4])

In [6]:
best_data = torch.zeros(13, 4)
for i, seed in enumerate(best_seeds):
    best_data[i] = results[i, seed]
best_data_advanced = results[torch.arange(13), best_seeds]
best_data_advanced

tensor([[1.0000e+00, 1.0000e+00, 5.5201e-04, 5.5229e-04],
        [9.7581e-01, 8.8608e-01, 7.2599e-04, 7.3007e-04],
        [9.7638e-01, 9.6203e-01, 5.9318e-04, 5.9093e-04],
        [7.4016e-01, 7.4684e-01, 6.5931e-04, 6.5930e-04],
        [8.8189e-01, 9.8734e-01, 7.5211e-04, 7.5404e-04],
        [9.2913e-01, 9.1139e-01, 8.0990e-04, 8.1391e-04],
        [7.5862e-01, 6.4516e-01, 6.4812e-04, 6.4901e-04],
        [8.1890e-01, 8.3544e-01, 6.0098e-04, 6.0092e-04],
        [5.9843e-01, 4.9367e-01, 2.4899e-03, 2.4934e-03],
        [9.2683e-01, 9.0698e-01, 7.2935e-04, 7.1571e-04],
        [9.6063e-01, 8.4810e-01, 7.3566e-04, 7.3750e-04],
        [8.5246e-01, 8.4127e-01, 5.9427e-04, 5.9206e-04],
        [8.8525e-01, 8.0952e-01, 7.0689e-04, 7.0646e-04]],
       grad_fn=<IndexBackward>)

In [7]:
results_std = torch.std(results, dim=1)

In [8]:
results_std

tensor([[0.0000e+00, 0.0000e+00, 9.0285e-06, 9.3116e-06],
        [2.6153e-01, 2.3217e-01, 2.1745e-05, 2.2722e-05],
        [5.3146e-03, 1.0757e-02, 7.4644e-05, 7.1729e-05],
        [0.0000e+00, 0.0000e+00, 4.7953e-06, 4.7948e-06],
        [4.1175e-02, 7.1866e-02, 2.9123e-05, 2.8307e-05],
        [5.0493e-02, 7.7066e-02, 1.0743e-04, 1.0810e-04],
        [0.0000e+00, 0.0000e+00, 4.9044e-05, 6.3271e-05],
        [5.5056e-03, 7.1854e-03, 1.2482e-05, 1.2296e-05],
        [1.1309e-01, 1.1096e-01, 2.7060e-04, 2.7005e-04],
        [1.7254e-01, 7.4354e-02, 2.2728e-04, 1.7415e-04],
        [3.2835e-02, 4.9134e-02, 1.3929e-04, 1.4624e-04],
        [8.6401e-03, 7.6674e-03, 1.3113e-05, 1.2366e-05],
        [1.5552e-02, 2.9173e-02, 2.2433e-05, 2.3280e-05]],
       grad_fn=<StdBackward>)

In [9]:
final = torch.cat([best_data_advanced, results_std], dim=1)
final.shape

torch.Size([13, 8])

In [10]:
np.savetxt('./result.txt', final.detach().numpy(), fmt='%.9f', delimiter='\t')